### This notebook is Initial submission for the hackathon 

https://datahack.analyticsvidhya.com/contest/women-in-the-loop-a-data-science-hackathon-by-bain/


In [5]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor
import lightgbm as lgbm
from sklearn import tree

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test_QkPvNLx.csv')
sample_submission = pd.read_csv('data/sample_submission_pn2DrMq.csv')

In [8]:
train_df.shape

(512087, 11)

In [9]:
train_df.head(3)

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75


In [10]:
test_df.head(3)

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric
0,883,883,1,Development,Course,1,0,1,0.007
1,884,884,1,Development,Course,1,0,1,0.007
2,885,885,1,Development,Course,1,0,1,0.007


In [11]:
sample_submission.head(3)

,ID,Sales
0,883,5
1,884,5
2,885,5


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 11 columns):
ID                    512087 non-null int64
Day_No                512087 non-null int64
Course_ID             512087 non-null int64
Course_Domain         512087 non-null object
Course_Type           512087 non-null object
Short_Promotion       512087 non-null int64
Public_Holiday        512087 non-null int64
Long_Promotion        512087 non-null int64
User_Traffic          512087 non-null int64
Competition_Metric    510323 non-null float64
Sales                 512087 non-null int64
dtypes: float64(1), int64(8), object(2)
memory usage: 43.0+ MB


In [13]:
train_df.Course_ID.nunique()

600

In [14]:
def get_course_catalog(train_df):
    temp_df = train_df.copy()
    temp_df['Day_No'] = train_df['Day_No'].mod(365)
    course_user_traffic_catalog_1 =temp_df.groupby(['Course_ID','Day_No'], as_index=False).agg({
        'User_Traffic': 'mean'  
    }).rename(columns = {'User_Traffic':'User_Traffic_Mean'})
    course_user_traffic_catalog_2 =temp_df.groupby(['Course_ID','Day_No'], as_index=False).agg({
        'User_Traffic': 'std'  
    }).rename(columns = {'User_Traffic':'User_Traffic_STD'})
    course_user_traffic_catalog_3 =temp_df.groupby(['Course_ID','Day_No'], as_index=False).agg({
        'User_Traffic': 'last'  
    }).rename(columns = {'User_Traffic':'User_Traffic_Recent'})
    course_user_traffic_catalog = pd.merge(course_user_traffic_catalog_1,course_user_traffic_catalog_2,
                                           on=['Course_ID','Day_No']).merge(course_user_traffic_catalog_3,
                                                                           on=['Course_ID','Day_No'])
    return course_user_traffic_catalog

In [15]:
def overall_preprocessing(df, course_user_traffic_catalog, is_test=False):
    df['Competition_Metric'] = df['Competition_Metric'].fillna(0.0)
    course_type = pd.get_dummies(df['Course_Type'])
    course_domain = pd.get_dummies(df['Course_Domain'])
    course_ids = pd.get_dummies(df['Course_ID'], prefix ='id')
    df_processed = pd.concat([df, course_type, course_domain, course_ids], axis=1)
    df_processed['Day_No'] = df_processed['Day_No'].mod(365)
    df_processed = df_processed.merge(course_user_traffic_catalog,
                                      on=['Course_ID','Day_No']).sort_values('ID').reset_index(drop=True)
    df_processed = df_processed.drop(columns = ['Course_ID','Course_Type','Course_Domain'])
    if is_test:
        return df_processed
    else:
        target = df_processed[['Sales']]
        df_processed = df_processed.drop(columns = ['Sales','User_Traffic'])
        return df_processed, target

# def course_domain_preprocessing(df, course_domain, is_test=False):
#     df_domain = df.loc[df['Course_Domain']==course_domain].reset_index()
#     if is_test:
#         return 'a'
#     else:
#         return 'b'

In [16]:
course_user_traffic_catalog = get_course_catalog(train_df)

In [17]:
df_train, df_target = overall_preprocessing(train_df, course_user_traffic_catalog)
df_pred = overall_preprocessing(test_df,course_user_traffic_catalog, True)

In [22]:
df_train.fillna(0, inplace=True)

In [18]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

clf = tree.DecisionTreeClassifier(random_state=0)
clf = clf.fit(df_train[:100], df_target[:100])

plt.figure()
tree.plot_tree(clf)
plt.show()


In [29]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_squared_log_error

def rmsle_cv(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=90).get_n_splits(df_train.values)
    rmse= np.sqrt(-cross_val_score(model, df_train.values, df_target.values, scoring="neg_mean_squared_log_error", cv = kf))*1000
    print(rmse)
    return(rmse)

In [30]:
lgbmr = lgbm.LGBMRegressor(n_estimators=130,min_child_samples=15,learning_rate=0.05)

In [31]:
lgbmr_rmse_e130_lr_05 = rmsle_cv(lgbmr)
#[150.51512858 146.26601396 177.54153857 171.50355708 182.38828315]

[232.23785117 235.24440153 257.52666559 250.51647377 238.62830037]


In [32]:
lgbmr = lgbm.LGBMRegressor(n_estimators=150,learning_rate=0.01)
lgbmr_rmse_e150_lr_01 = rmsle_cv(lgbmr)

[278.93554444 293.65844911 302.73317841 303.96682913 278.9933543 ]


In [33]:
lgbmr = lgbm.LGBMRegressor(n_estimators=130,num_leaves=600, min_child_samples=15,learning_rate=0.05)
lgbmr_rmse_e130_lr_05 = rmsle_cv(lgbmr)
#[150.51512858 146.26601396 177.54153857 171.50355708 182.38828315]

[222.40336621 220.54482047 245.62764142 238.25554758 241.97693789]


In [34]:
lgbmr = lgbm.LGBMRegressor(n_estimators=200,num_leaves=600, min_child_samples=15,learning_rate=0.05)
lgbmr_rmse_e200_lr_05 = rmsle_cv(lgbmr)
#[150.51512858 146.26601396 177.54153857 171.50355708 182.38828315]

[221.75434838 218.77216166 243.66876874 235.82743558 241.33928382]


In [39]:
lgbmr = lgbm.LGBMRegressor(n_estimators=200,num_leaves=600, min_child_samples=200,learning_rate=0.05)
lgbmr_rmse_e180_lr_1 = rmsle_cv(lgbmr)
#[150.51512858 146.26601396 177.54153857 171.50355708 182.38828315]

[220.56736151 219.90285194 246.88128284 234.11320698 240.50439324]


In [41]:
df_pred_ID = df_pred['ID']

In [42]:
lgbmr.fit(df_train.drop(columns=['ID']),df_target)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=200, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=200, n_jobs=-1, num_leaves=600, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [45]:
df_pred.to_csv()

Sales    [126.4148551095078, 115.59652529146469, 126.75...
dtype: object

In [47]:
def save_submission(df_pred_ID, prediction, filename):
    result = pd.concat([df_pred_ID,pd.DataFrame({'Sales':prediction})],axis=1)
    result.to_csv('submissions/' + filename + '.csv', index=False)

In [48]:
save_submission(df_pred_ID, lgbmr.predict(df_pred.drop(columns=['ID'])),'year_mod_single_model_lgbm_e200_lr05_cs200')

In [50]:
sample_submission.head(2)

,ID,Sales
0,883,5
1,884,5


In [51]:
pd.read_csv('submissions/year_mod_single_model_lgbm_e200_lr05_cs200.csv').head(2)

,ID,Sales
0,883,126.414855
1,884,115.596525


In [ ]:
lgbmr.fit()